In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate,Dense,Flatten,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#Loading the Image and mask information paths
im_path = "/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Train/images"
mask_path = "/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Train/masks"

im_list = []
for filename in os.listdir(im_path):
    if filename.endswith((".jpg")):  # Add more image extensions if needed
        # Construct the full path to the image
        im_list.append(os.path.join(im_path, filename))
im_list.sort()     

mask_list = []
for filename in os.listdir(mask_path):
    if filename.endswith((".png")):  # Add more image extensions if needed
        # Construct the full path to the image
        mask_list.append(os.path.join(mask_path, filename))
mask_list.sort()

In [5]:
# Create a CNN model equivalent to the VGG-16.
model1 = Sequential()
# Block 1
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(112,112,3)))
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model1.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model1.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model1.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2), strides=(2, 2)))

#Middle
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))

# Flatten
model1.add(Flatten())

# Fully connected layers
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(7, activation='softmax'))
# Compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 28, 28, 64)        0         
 g2D)                                                   

In [7]:
#Image pixel data loading as we resize it to (112,112,3)
im_info = []
for j in range(len(im_list)):
    im_info.append(cv2.resize(cv2.imread(im_list[j]), (112, 112)))
im_info = np.array(im_info)/255

label_info = pd.read_csv("/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Train/GroundTruth.csv")
label_info = label_info.iloc[:,1:].values

from sklearn.utils import shuffle
im_info,label_info = shuffle(im_info,label_info)
im_test_info = im_info[8000:]
label_test_info = label_info[8000:]
im_info = im_info[:8000]
label_info = label_info[:8000]

In [9]:
#Model Training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/kaggle/working/main_model.h5', monitor='val_loss', save_best_only=True)
history = model1.fit(im_info,label_info, epochs=100,batch_size=64, validation_data=(im_test_info,label_test_info),callbacks=[early_stopping,model_checkpoint])

Epoch 1/100
125/125 [==============================] - 28s 77ms/step - loss: 1.1860 - accuracy: 0.6352 - val_loss: 0.9737 - val_accuracy: 0.6779
Epoch 2/100
  2/125 [..............................] - ETA: 7s - loss: 0.9858 - accuracy: 0.6797

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


125/125 [==============================] - 8s 64ms/step - loss: 1.0109 - accuracy: 0.6671 - val_loss: 0.9205 - val_accuracy: 0.6779
Epoch 3/100
125/125 [==============================] - 8s 64ms/step - loss: 0.9585 - accuracy: 0.6671 - val_loss: 0.8881 - val_accuracy: 0.6779
Epoch 4/100
125/125 [==============================] - 8s 63ms/step - loss: 0.9312 - accuracy: 0.6700 - val_loss: 1.0272 - val_accuracy: 0.6868
Epoch 5/100
125/125 [==============================] - 8s 64ms/step - loss: 0.9154 - accuracy: 0.6711 - val_loss: 0.8784 - val_accuracy: 0.6948
Epoch 6/100
125/125 [==============================] - 8s 63ms/step - loss: 0.8987 - accuracy: 0.6776 - val_loss: 0.8793 - val_accuracy: 0.6933
Epoch 7/100
125/125 [==============================] - 8s 65ms/step - loss: 0.8759 - accuracy: 0.6855 - val_loss: 0.8417 - val_accuracy: 0.7012
Epoch 8/100
125/125 [==============================] - 8s 65ms/step - loss: 0.8532 - accuracy: 0.6844 - val_loss: 0.7989 - val_accuracy: 0.7067
Epoc

In [10]:
#Model downloading
model1.save('/kaggle/working/main_model.h5')
import os 
os.chdir(r'/kaggle/working') 
from IPython.display import FileLink 
FileLink(r'main_model.h5')

/kaggle/working/main_model.h5

In [12]:
#Loading the test data
im_test_path = "/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISIC2018_Task3_Test_Input/"
label_test_info = pd.read_csv("/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv")
label_test_info = label_test_info.iloc[:,1:]

im_test_list = []
for filename in os.listdir(im_test_path):
    if filename.endswith((".jpg")):  # Add more image extensions if needed
        # Construct the full path to the image
        im_test_list.append(os.path.join(im_test_path, filename))
im_test_list.sort()     

In [13]:
#Resizing the test data
im_test_info = []
for j in range(len(im_test_list)):
    im_test_info.append(cv2.resize(cv2.imread(im_test_list[j]), (112, 112)))
im_test_info = np.array(im_test_info)/255

In [14]:
#Using test data to run the model
im_class_prediction = model1.predict(im_test_info)

48/48 [==============================] - 1s 13ms/step


In [16]:
from tensorflow.keras.utils import to_categorical
im_class_prediction = [to_categorical(np.argmax(i), num_classes=7) for i in im_class_prediction]

In [19]:
im_class_prediction=np.array(im_class_prediction)

In [15]:
from tensorflow.keras import backend as K

def normalized_multi_class_accuracy(y_true_info, y_prediction_info):
    true_pos = K.sum(K.round(K.clip(y_true_info * y_prediction_info, 0, 1)), axis=0)
    true_neg = K.sum(K.round(K.clip((1 - y_true_info) * (1 - y_prediction_info), 0, 1)), axis=0)
    total_samples = K.sum(K.round(K.clip(y_true_info + (1 - y_true_info), 0, 1)), axis=0)

    class_acc = (true_pos + true_neg) / (total_samples + K.epsilon())
    normalized_acc = K.mean(class_acc)

    return normalized_acc

In [23]:
#Normalized multiclass accuracy for classification model
normalized_multi_class_accuracy(label_test_info,im_class_prediction).numpy()*100

90.0415721784742

# **Model training for image segmentation**

In [25]:
#Developing an unet model for image segmentation.

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def seg_unet_model(input_shape=(112, 112, 3), num_classes=7):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(32, 3, activation='relu', padding='same', trainable=False)(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same', trainable=False)(conv1)
    pooling1 = MaxPooling2D(pool_size=(2, 2), trainable=False)(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same', trainable=False)(pooling1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same', trainable=False)(conv2)
    pooling2 = MaxPooling2D(pool_size=(2, 2), trainable=False)(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same', trainable=False)(pooling2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same', trainable=False)(conv3)
    conv3 = Conv2D(128, 3, activation='relu', padding='same', trainable=False)(conv3)
    pooling3 = MaxPooling2D(pool_size=(2, 2), trainable=False)(conv3)

    # Bottom layer
    conv4 = Conv2D(32, 3, activation='relu', padding='same', trainable=False)(pooling3)
    conv4 = Conv2D(32, 3, activation='relu', padding='same', trainable=False)(conv4)

    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = concatenate([conv3,up5], axis=-1)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(conv5)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    concat6 = concatenate([conv2,up6], axis=-1)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(concat6)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    concat7 = concatenate([conv1,up7], axis=-1)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(concat7)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(conv7)
    
    # Output layer
    outputs = Conv2D(3, 1, activation='sigmoid')(conv7)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return model

# Create U-Net model
segmentation_model = seg_unet_model()

# Print the model summary
segmentation_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 112, 112, 3)]        0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 112, 112, 32)         896       ['input_2[0][0]']             
                                                                                                  
 conv2d_27 (Conv2D)          (None, 112, 112, 32)         9248      ['conv2d_26[0][0]']           
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 56, 56, 32)           0         ['conv2d_27[0][0]']           
 g2D)                                                                                       

In [27]:
#Loading the previous classification model
main_model=load_model("/kaggle/input/main-model/main_model.h5")

In [28]:
#Transfer Learning involves fixing the weights of non-trainable layers.
segmentation_model.layers[1].set_weights(main_model.layers[0].get_weights())
segmentation_model.layers[2].set_weights(main_model.layers[1].get_weights())
segmentation_model.layers[4].set_weights(main_model.layers[3].get_weights())
segmentation_model.layers[5].set_weights(main_model.layers[4].get_weights())
segmentation_model.layers[7].set_weights(main_model.layers[6].get_weights())
segmentation_model.layers[8].set_weights(main_model.layers[7].get_weights())
segmentation_model.layers[9].set_weights(main_model.layers[8].get_weights())
segmentation_model.layers[11].set_weights(main_model.layers[10].get_weights())
segmentation_model.layers[12].set_weights(main_model.layers[11].get_weights())

In [29]:
#Loading and resizing the image data and mask data
im_info = []
mask_info = []
for j in range(len(im_list)):
    im_info.append(cv2.resize(cv2.imread(im_list[j]), (112, 112)))
    mask_info.append(cv2.resize(cv2.imread(mask_list[j]), (112, 112)))

In [30]:
im_info = np.array(im_info)/255
mask_info = np.array(mask_info)/255

from sklearn.utils import shuffle
im_info,mask_info = shuffle(im_info,mask_info)
im_test_info = im_info[8000:]
mask_test = mask_info[8000:]
im_info = im_info[:8000]
mask_info = mask_info[:8000]

In [31]:
#Model training
segmentation_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = segmentation_model.fit(im_info,mask_info, epochs=100,batch_size=64, validation_data=(im_test_info,mask_test),callbacks=[early_stopping])

Epoch 1/100
125/125 [==============================] - 28s 179ms/step - loss: 0.2886 - accuracy: 0.3316 - val_loss: 0.2149 - val_accuracy: 0.1718
Epoch 2/100
125/125 [==============================] - 20s 159ms/step - loss: 0.1996 - accuracy: 0.2537 - val_loss: 0.1822 - val_accuracy: 0.2668
Epoch 3/100
125/125 [==============================] - 20s 160ms/step - loss: 0.1763 - accuracy: 0.2740 - val_loss: 0.1898 - val_accuracy: 0.3661
Epoch 4/100
125/125 [==============================] - 20s 162ms/step - loss: 0.1668 - accuracy: 0.2835 - val_loss: 0.1613 - val_accuracy: 0.2372
Epoch 5/100
125/125 [==============================] - 20s 164ms/step - loss: 0.1585 - accuracy: 0.2918 - val_loss: 0.1601 - val_accuracy: 0.3540
Epoch 6/100
125/125 [==============================] - 21s 164ms/step - loss: 0.1577 - accuracy: 0.3327 - val_loss: 0.1612 - val_accuracy: 0.3417
Epoch 7/100
125/125 [==============================] - 21s 165ms/step - loss: 0.1514 - accuracy: 0.3496 - val_loss: 0.1613 -

In [33]:
#Observing test data paths
im_test_path = "/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISBI2016_ISIC_Part1_Test_Data/"
mask_test_path = "/kaggle/input/cs770-final-project-skin-lesion-segmentation/Project_Data/Test/ISBI2016_ISIC_Part1_Test_GroundTruth"

im_test_list = []
for filename in os.listdir(im_test_path):
    if filename.endswith((".jpg")):  # Add more image extensions if needed
        # Construct the full path to the image
        im_test_list.append(os.path.join(im_test_path, filename))
im_test_list.sort()   

mask_test_list = []
for filename in os.listdir(mask_test_path):
    if filename.endswith((".png")):  # Add more image extensions if needed
        # Construct the full path to the image
        mask_test_list.append(os.path.join(mask_test_path, filename))
mask_test_list.sort()   

In [34]:
#Loading and resizing the test data 
im_test_info = []
for j in range(len(im_test_list)):
    im_test_info.append(cv2.resize(cv2.imread(im_test_list[j]), (112, 112)))
im_test_info = np.array(im_test_info)/255

mask_test_info = []
for j in range(len(mask_test_list)):
    mask_test_info.append(cv2.resize(cv2.imread(mask_test_list[j]), (112, 112)))
mask_test_info = np.array(mask_test_info)/255

In [35]:
#Prediction based on test data 
mask_predictions = segmentation_model.predict(im_test_info)

12/12 [==============================] - 2s 115ms/step


In [36]:
#Converting into 0,1
mask_predictions = (mask_predictions > 0.5).astype(int)

In [37]:
#Jaccard_Index
def jaccard_index(y_true_info, y_prediction_info, smooth=1.0):
    intersection = np.sum(y_true_info * y_prediction_info)
    union = np.sum(y_true_info) + np.sum(y_prediction_info) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

In [38]:
#calculation of Jaccard Index 
iou_metric = jaccard_index(mask_test_info,mask_predictions)

In [39]:
iou_metric

0.7628783917533198